In [1]:
import pandas as pd
import os
import torch
device = torch.device("cuda:0")

path = os.getcwd()
data = 'top_rated_wines.csv'
df = pd.read_csv(os.path.join(path, data))
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index

In [2]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L6-v2', device = "cuda:0") # Model to create embeddings


c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
from qdrant_client import models, QdrantClient
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\lol_a\AppData\Local\Temp\ipykernel_11096\866372840.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [4]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [6]:
# Search time for awesome wines!
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Bodega Colome Altura Maxima Malbec 2012', 'region': 'Salta, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Winemaker Thibaut Delmotte has crafted wines of distinction and international acclaim for Colome. He believes the Malbec from Altura Maxima Vineyard is the embodiment of two extremes - a traditional grape variety from his French origins made from the vineyard that challenges all convention in the modern viticultural world.'} score: 0.6179682022766719
{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s 

In [7]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [11]:
#RAG
prompt = f'''
    {search_results} \n

    {user_prompt}
'''

print(prompt)


    [{'name': 'Bodega Colome Altura Maxima Malbec 2012', 'region': 'Salta, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Winemaker Thibaut Delmotte has crafted wines of distinction and international acclaim for Colome. He believes the Malbec from Altura Maxima Vineyard is the embodiment of two extremes - a traditional grape variety from his French origins made from the vineyard that challenges all convention in the modern viticultural world.'}, {'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s ability to defer im

In [12]:

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(respuesta)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    [{'name': 'Bodega Colome Altura Maxima Malbec 2012','region': 'Salta, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Winemaker Thibaut Delmotte has crafted wines of distinction and international acclaim for Colome. He believes the Malbec from Altura Maxima Vineyard is the embodiment of two extremes - a traditional grape variety from his French origins made from the vineyard that challenges all convention in the modern viticultural world.'}, {'name': 'Catena Zapata Adrianna Vineyard Malbec 2004','region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s ability to defer imme